In [2]:
from pyspark import SparkContext as sc
from pyspark.sql import SparkSession as ss

In [3]:
sc = sc()
ss = ss(sc)

In [4]:
sup1 = ss.read.csv('file:///etc/dataset/superhero.csv', inferSchema=True, header=True)

In [6]:
sup1.describe()

DataFrame[summary: string, _c0: string, name: string, Gender: string, Eye color: string, Race: string, Hair color: string, Height: string, Publisher: string, Skin color: string, Alignment: string, Weight: string]

In [21]:
new_df = sup1.withColumnRenamed('Eye color', 'eye_color').withColumnRenamed('Hair color', 'hair_color').withColumnRenamed('Skin color', 'skin_color')

In [22]:
new_df.describe()

DataFrame[summary: string, _c0: string, name: string, Gender: string, eye_color: string, Race: string, hair_color: string, Height: string, Publisher: string, skin_color: string, Alignment: string, Weight: string]

In [23]:
new_df.write.parquet('/etc/dataset/superhero.parquet')

In [24]:
from_parquet = ss.read.parquet('file:///etc/dataset/superhero.parquet')

In [25]:
from_parquet.describe()

DataFrame[summary: string, _c0: string, name: string, Gender: string, eye_color: string, Race: string, hair_color: string, Height: string, Publisher: string, skin_color: string, Alignment: string, Weight: string]

In [28]:
from_parquet.columns

['_c0',
 'name',
 'Gender',
 'eye_color',
 'Race',
 'hair_color',
 'Height',
 'Publisher',
 'skin_color',
 'Alignment',
 'Weight']

In [29]:
for column in from_parquet.columns:
    from_parquet = from_parquet.withColumnRenamed(column, column.lower().replace(' ', '_'))

In [30]:
from_parquet.describe()

DataFrame[summary: string, _c0: string, name: string, gender: string, eye_color: string, race: string, hair_color: string, height: string, publisher: string, skin_color: string, alignment: string, weight: string]

In [36]:
from pyspark.ml.feature import StringIndexer

In [31]:
from pyspark.ml.feature import OneHotEncoder

In [37]:
to_parse = ['gender', 'eye_color', 'race', 'hair_color', 'publisher', 'skin_color', 'alignment']

In [44]:
from_parquet = from_parquet.dropna()

In [45]:
indexer = StringIndexer(inputCols=to_parse, outputCols=list(map(lambda x: x+'index', to_parse)))

In [46]:
indexed = indexer.fit(from_parquet).transform(from_parquet)

In [47]:
indexed.describe()

DataFrame[summary: string, _c0: string, name: string, gender: string, eye_color: string, race: string, hair_color: string, height: string, publisher: string, skin_color: string, alignment: string, weight: string, genderindex: string, hair_colorindex: string, skin_colorindex: string, raceindex: string, eye_colorindex: string, publisherindex: string, alignmentindex: string]

In [52]:
indexed.count()

719

In [54]:
encoder = OneHotEncoder().setInputCols(list(map(lambda x: x+'index', to_onehot))).setOutputCols(list(map(lambda x: x+'vec', to_parse)))

In [55]:
model = encoder.fit(indexed)

In [56]:
done = model.transform(indexed)

In [58]:
done.describe()

DataFrame[summary: string, _c0: string, name: string, gender: string, eye_color: string, race: string, hair_color: string, height: string, publisher: string, skin_color: string, alignment: string, weight: string, genderindex: string, hair_colorindex: string, skin_colorindex: string, raceindex: string, eye_colorindex: string, publisherindex: string, alignmentindex: string]